In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

##### GAN 개념 참고 
https://ddongwon.tistory.com/124
https://velog.io/@a01152a/%EB%85%BC%EB%AC%B8-%EC%9D%BD%EA%B8%B0-%EB%B0%8F-%EA%B5%AC%ED%98%842-GAN


- 두개의 모델이 adversarial process를 통해 동작하며 학습 수행 -> generative model을 estimate하는 새로운 프레임워크
- 동시에 2가지 모델을 학습 (1) Generative model "G" (2) Discriminative model "D"
- G와 D 모델이 multilayer perceptron 구조를 지니고 있다면, backpropagation을 통한 학습 가능함.

[Discriminator]
- D는 입력된 이미지가 진짜(1)인지 가짜(0)인지를 구분

[Generator]
- G는 확률분포 Pg가 실제 data X의 확률분포를 닮도록 만드는 것이 목표.
- G는 latent vector z를 입력받아 가짜 data를 생성해내는 기능 수행
- G는 최대한 D가 실수를 하도록 하는 것이 목표로, 모델 D가 모델 G가 생성한 데이터와 실 데이터를 구분하지 못하도록하는 것이 목표


[이전 모델] 기존의 generative model은 다양한 확률적 계산 이슈가 존재하였기 때문에 좋은 성능을 기대하기 어려웠으며, Markov chains이나 inference network와 같은 복잡한 기능이 필요하다는 문제점이 존재하였음.
본 모델은 해당 한계를 보완

- loss function 
![nn](image/loss_function.png)

[좌변] 실제 데이터 x에 대하여 log D(x) 값의 기댓값, D의 성능이 좋을 수록 증가

[우변] 가짜 데이터 z에 대하여 log(1-D(G(z))) 값의 기댓값, D가 성능이 좋을 수록 증가

따라서, D는 V(D, G) 함수를 maximize, G는 V(D, G) 함수를 minimize 하고자 함.

In [6]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid() # 0.5를 기준으로 진짜와 가짜를 classification
        )
    
    def forward(self, x): # 모델이 학습데이터 x를 입력받아서 forward propagation을 진행시키는 함수
        return self.disc(x) 

In [7]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim): 
        super(Generator, self).__init__()

        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh() # -1~ 1 사이값 출력
        )
    
    def forward(self, x):
        return self.gen(x)

In [3]:
disc = Discriminator(784)